### Importing Necessary Libraries

In [1]:
import cv2
from pylab import array
import sys
from matplotlib import pyplot as plt
import numpy as np

### Network Initialization

COCO Output Format Nose – 0, Neck – 1, Right Shoulder – 2, Right Elbow – 3, Right Wrist – 4, Left Shoulder – 5, Left Elbow – 6, Left Wrist – 7, Right Hip – 8, Right Knee – 9, Right Ankle – 10, Left Hip – 11, Left Knee – 12, LAnkle – 13, Right Eye – 14, Left Eye – 15, Right Ear – 16, Left Ear – 17, Background – 18

MPII Output Format Head – 0, Neck – 1, Right Shoulder – 2, Right Elbow – 3, Right Wrist – 4, Left Shoulder – 5, Left Elbow – 6, Left Wrist – 7, Right Hip – 8, Right Knee – 9, Right Ankle – 10, Left Hip – 11, Left Knee – 12, Left Ankle – 13, Chest – 14, Background – 15

In [66]:
MODE = "MPI"

if MODE is "COCO":
    protoFile = "models/pose/coco/pose_deploy_linevec.prototxt"
    weightsFile = "models/pose/coco/pose_iter_440000.caffemodel"
    nPoints = 18
    POSE_PAIRS = [ [1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]

elif MODE is "MPI" :
    protoFile = "models/pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt"
    weightsFile = "models/pose/mpi/pose_iter_160000.caffemodel"
    nPoints = 15
    POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13] ]

net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
net.setPreferableBackend(cv2.dnn.DNN_TARGET_CPU)
print("Using CPU device")

<>:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\kevin\AppData\Local\Temp\ipykernel_3456\2187113547.py:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if MODE is "COCO":
C:\Users\kevin\AppData\Local\Temp\ipykernel_3456\2187113547.py:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif MODE is "MPI" :


Using CPU device


### Testing camera

In [70]:
# Testing camera
cap = cv2.VideoCapture(1, cv2.CAP_DSHOW)

# cap.set(cv2.CAP_PROP_EXPOSURE, -8.0)
print(cap)

while(True):

    # Capture the video frame
    # by frame
    ret, frame = cap.read()

    # Display the resulting frame
    cv2.imshow('frame', frame)

    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop release the cap object
cap.release()
# Destroy all the windows
cv2.destroyAllWindows()

< cv2.VideoCapture 0000024734B83D70>


### Reading in the frames

In [71]:
import time
time.sleep(4)
# Initial video reading
FRAMES_READ = 5  # Assume 30 fps
FRAMES = []

cap = cv2.VideoCapture(1, cv2.CAP_DSHOW)  # Apparently dshow makes it faster
for i in range(FRAMES_READ):
    ret, frame = cap.read()
    FRAMES.append(frame)
    cv2.imshow("frame", frame)
    cv2.setWindowProperty("frame", cv2.WND_PROP_TOPMOST, 1)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop release the cap object
cap.release()
# Destroy all the windows
cv2.destroyAllWindows()
print(len(FRAMES), array(FRAMES).nbytes/(1024*1024))

5 4.39453125


### Pose Processing

In [72]:
loop = []
for frame in FRAMES:
    frameCopy = np.copy(frame)
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    threshold = 0.1

    inWidth = 500
    inHeight = 500
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                            (0, 0, 0), swapRB=False, crop=False)
    net.setInput(inpBlob)
    # Bottleneck

    output = net.forward()
    H = output.shape[2]
    W = output.shape[3]

    # Empty list to store the detected keypoints
    points = []

    for i in range(nPoints):
        # confidence map of corresponding body's part.
        probMap = output[0, i, :, :]

        # Find global maxima of the probMap.
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        # Scale the point to fit on the original image
        x = (frameWidth * point[0]) / W
        y = (frameHeight * point[1]) / H

        if prob > threshold :
            cv2.circle(frameCopy, (int(x), int(y)), 4, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)

            # Add the point to the list if the probability is greater than the threshold
            points.append((int(x), int(y)))
        else :
            points.append(None)

    # Draw Skeleton
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        # if points[partA] and points[partB]:
        #     cv2.line(frame, points[partA], points[partB], (0, 255, 255), 2)
        #     cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

    loop.append(frameCopy)
    # cv2.imshow('Output-Keypoints', frameCopy)
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break
    print("processing...")
print("done")



processing...
processing...
processing...
processing...
processing...
done


### Looping the image

In [75]:
# for i in range(10):
# for j in loop[:5]:
    # print(j)
# print(loop)
# import time
for _ in range(1):
    for frame in loop:
        cv2.imshow("frame", frame)
        cv2.waitKey(0)
        # cv2.setWindowProperty("frame", cv2.WND_PROP_TOPMOST, 1)
        print("bruh")
        # time.sleep(2)
        # cv2.imshow("frame", loop)
        # print("bruh2")

cv2.waitKey(0)
cv2.destroyAllWindows()

bruh
bruh
bruh
bruh
bruh


In [51]:
cv2.imshow("bruh", FRAMES[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [65]:
# len(loop)
print(nPoints)

18
